In [1]:
import sys
import torch
import tntorch as tn
import numpy as np
sys.path.append('..')
import lib

In [2]:
rho, train_X, train_y = lib.read_data('../data/2020_01_29_stabilizer_dim32_0.dat')
dim = rho.shape[0]
n_qubits = np.log2(dim)

tensor_rank = 2**n_qubits
batch_size = 100
lr = 1e-3

sigma = lib.simulator.randomMixedState(rho.shape[0])
sigma_real, sigma_imag = [tn.Tensor(x, ranks_tt=tensor_rank, requires_grad=True) for x in [np.real(sigma), np.imag(sigma)]]

epoches = 10

In [3]:
def trace(tensor):
    if len(tensor.shape) == 2:
        return sum([tensor[i,i] for i in range(tensor.shape[0])])
    if len(tensor.shape) == 3:
        return sum([tensor[:, i,i] for i in range(tensor.shape[1])])

def cholesky(sigma_real, sigma_imag):
    sigma_real, sigma_imag = sigma_real.dot(sigma_real, k=1)+sigma_imag.dot(sigma_imag, k=1), sigma_real.dot(sigma_imag, k=1)-sigma_imag.dot(sigma_real, k=1)
    trace = sum([sigma_real[i,i] for i in range(rho.shape[0])])
    sigma_real, sigma_imag = [x/trace for x in [sigma_real, sigma_imag]]
    return sigma_real, sigma_imag
    
#initial_trace = sum((np.real(trace(test_X.dot(sigma)))-test_y)**2)

def loss(sigma_real, sigma_imag):
    sigma_real, sigma_imag = cholesky(sigma_real, sigma_imag)
    res = 0
    idx = np.random.choice(np.arange(train_X.shape[0]), batch_size)
    for E_m,y_m in zip(train_X[idx], train_y[idx].astype('float64')):
        E_real, E_imag = [tn.Tensor(x) for x in [np.real(E_m), np.imag(E_m)]]
        res += ((E_real.dot(sigma_real)+E_imag.dot(sigma_imag)-y_m)**2)
#     return res/(initial_trace*train_X.shape[0])
    return res
    

def eval_loss(sigma_real, sigma_imag): # any score function can be used here
    sigma_real_, sigma_imag_ = cholesky(sigma_real, sigma_imag)
    sigma = sigma_real_.torch().detach().cpu().numpy() + 1j*sigma_imag_.torch().detach().cpu().numpy()
    return -lib.fidelity(sigma, rho)

#print('Trace before: %f'%initial_trace)

lib.tn_optimize([sigma_real, sigma_imag], loss, eval_loss, tol=0, patience=300,print_freq=10,lr=lr)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
iter: 0 |  loss: 53.23578643798828 | eval_loss: -0.04496200178681223 | total time: 0.2689054012298584
iter: 10 |  loss: 52.315093994140625 | eval_loss: -0.04493150956139027 | total time: 3.5881080627441406
iter: 20 |  loss: 52.11803436279297 | eval_loss: -0.04458478097192715 | total time: 7.150235176086426
iter: 30 |  loss: 51.34756851196289 | eval_loss: -0.04394072379661496 | total time: 10.091796398162842
iter: 40 |  loss: 53.029273986816406 | eval_loss: -0.042155049215325334 | total time: 13.357889413833618
iter: 50 |  loss: 44.427574157714844 | eval_loss: -0.04092186272900992 | total time: 17.000505447387695
iter: 60 |  loss: 53.47964096069336 | eval_loss: -0.03983913484005695 | total time: 20.562462329864502
iter: 70 |  loss: 51.585205078125 | eval_loss: -0.037769683876909654 | total time: 23.90771174430847
iter: 80 |  loss: 50.76714324951172 | eval_loss: -0.03666214583012489 | tot